In [2]:
%load_ext autoreload
%autoreload 2
import xarray as xr
from matplotlib import pyplot as plt
import numpy as np
#set path to import my_toools.py
import sys
sys.path.append('/home/onno/Thesis/Scripts')
import my_tools
from my_tools import file_dic, plot_dic
import pandas as pd
import glob
from mpl_toolkits.basemap import Basemap, addcyclic
from matplotlib.patches import Polygon



%matplotlib qt



In [ ]:
#set path change accordingly
path = '/media/onno/Algemeen/Thesis/GFS_histogram/'
path_file = '/media/onno/Algemeen/Thesis/'
#List specific variable data
variablez = ['phasespeed']
for variable in variablez:
    print(variable)
    file_gfs = file_dic[variable]['GFS']
    file_era = file_dic[variable]['ERA5']
    #open xarray datasets
    gfs = xr.open_dataset(path_file+file_gfs,decode_times=False)
    era = xr.open_dataset(path_file+file_era,decode_times=False)

    #change time values to normal datetime values
    gfs['time']=my_tools.convert_date_gefs_r(gfs.time.values)
    era['time']=my_tools.convert_date_era_r(era.time.values)
    if list(era.coords)[1]=='longitude':
        era=era.rename({'longitude':'lon'})
    if list(era.coords)[2]=='latitude':
        era=era.rename({'latitude':'lat'})

    #Select specific 24 by 24 degrees area 
#     latz = [(80,60),(60,40),(40,20)]
#     lonz = [(320,340),(290,310),(320,340)]
    latz = [(80,60),(60,40),(40,20)]
    lonz = [(320,340),(290,310),(320,340)]
    for i in range(len(latz)):
        print(latz[i])
        gfs_selected = gfs.sel(lat=slice(latz[i][0],latz[i][1]),
                              lon=slice(lonz[i][0],lonz[i][1]))
        era_selected = era.sel(lat=slice(latz[i][0],latz[i][1]),
                              lon=slice(lonz[i][0],lonz[i][1]))
        for j in range(11):
            if (variable=='phasespeed') & (j==0):
                continue
            print(j)

            #Select data for each day and specific lead time
            gfs_selected_d = gfs_selected.sel(time=[i for i in pd.date_range('1984-12-1','2019-11-{}'.format(30-j))],lead=j*24)
            #Calculate mean weighted average (see my_tools.py for more details)
            weighted_mean = my_tools.weighted_average_area_3D(gfs_selected_d,variable)
            #write data to netcdf file
            weighted_mean.to_netcdf(path+'/GFS_error_weighted_mean_{}_day_{:02d}_lat_lower_left_{}_lon_lower_left_{}_attempt2.nc'.format(variable,j,latz[i][1],lonz[i][0]))
#             weighted_mean.to_netcdf(path+'try.nc')

phasespeed
(80, 60)
1
2
3


In [8]:
#Seasonally split the timeseries that have been calculated in the 

#List all files
path = '/home/onno/Thesis/GFS_histogram/'
#Dictionary for seasonal data
seasonz = {'MAM':[3,4,5],
          'JJA':[6,7,8],
          'SON':[9,10,11],
          'DJF':[12,1,2]}
#Looop over all files
latz = [(80,60),(60,40),(40,20)]
lonz = [(320,340),(290,310),(320,340)]
variablez=['envelope','phasespeed','u_wind']
#loop over variables        
for k,variable in enumerate(variablez):
    print(variable)
    #loop over longitude latitude boxes
    for j in range(len(latz)):
        print(latz[j])
        #loop over each lead day
        for i in range(11):
            print(i)
            #because phasespeed data at lead day 0 is undefined we skip this
            if (variable=='phasespeed') & (i==0):
                continue
            #open file
            file_gen = 'GFS_error_weighted_mean_{}_day_{:02d}_lat_lower_left_{}_lon_lower_left_{}.nc'
            file = file_gen.format(variable,i,latz[j][1],lonz[j][0])
            ds = xr.open_dataset(path+file,decode_times=False)
            #set datetime data of dataset to normal datetime values by defining timestamp of first day in dataset
            init_time = pd.Timestamp('1984-12-01')
            ds['time'] = [init_time+pd.Timedelta(days=i) for i in ds.time.values]
            #loop over season
            for key in seasonz:
                monthz = (ds.time+pd.Timedelta(days=i)).dt.month
                #check if season is in monthz
                idxz = np.isin(monthz,seasonz[key])
                #select only timestamps that are present in season
                ds_season = ds.sel(time=idxz)
                #save to netcdf file. Use original filename without path and extension and add season name to it
                ds_season.to_netcdf(path+'/seasonal/{}_{}.nc'.format(file[:-3],key))                
        
    

envelope
(80, 60)
0
1
2
3
4
5
6
7
8
9
10
(60, 40)
0
1
2
3
4
5
6
7
8
9
10
(40, 20)
0
1
2
3
4
5
6
7
8
9
10
phasespeed
(80, 60)
0
1
2
3
4
5
6
7
8
9
10
(60, 40)
0
1
2
3
4
5
6
7
8
9
10
(40, 20)
0
1
2
3
4
5
6
7
8
9
10
u_wind
(80, 60)
0
1
2
3
4
5
6
7
8
9
10
(60, 40)
0
1
2
3
4
5
6
7
8
9
10
(40, 20)
0
1
2
3
4
5
6
7
8
9
10


In [107]:
path = '/media/onno/Algemeen/Thesis/GFS_histogram/seasonal/'
path_file = '/media/onno/Algemeen/Thesis/'
variablez = ['envelope','phasespeed','u_wind']
monthz = ['MAM','JJA','SON','DJF']
latz = [(80,60),(60,40),(40,20)]
lonz = [(320,340),(320,340),(320,340)]
dayz = [2,5,8]
y_limz = [1000,1000,750]
rangez = [(0,50),(-15,25),(-10,50)]
for j,variable in enumerate(variablez):
    for i in range(len(latz)):
        for month in monthz:
            fig,ax = plt.subplots(figsize=(16,9))
            ds_day2 = xr.open_dataset(path+'/GFS_error_weighted_mean_{}_day_02_lat_lower_left_{}_lon_lower_left_{}_{}.nc'.format(variable,latz[i][1],lonz[i][0],month),
                                     decode_times=False)
            ds_day5 = xr.open_dataset(path+'/GFS_error_weighted_mean_{}_day_05_lat_lower_left_{}_lon_lower_left_{}_{}.nc'.format(variable,latz[i][1],lonz[i][0],month),
                                     decode_times=False)
            ds_day8 = xr.open_dataset(path+'/GFS_error_weighted_mean_{}_day_08_lat_lower_left_{}_lon_lower_left_{}_{}.nc'.format(variable,latz[i][1],lonz[i][0],month),
                                     decode_times=False)
            ax.hist(x=[ds_day2[plot_dic[variable]['data_variable']].values,ds_day5[plot_dic[variable]['data_variable']].values,ds_day8[plot_dic[variable]['data_variable']].values],
                    bins=20,range=rangez[j],color=['purple','red','orange'],density=True)
            ax.legend(['Day 2','Day 5','Day 8'])
#             ax.set_ylim([0,y_limz[j]])
#             ax.set_ylabel('N')
            ax.set_xlabel(plot_dic[variable]['label'])
            fig.suptitle('GEFS Distribution of {} at {}-{} N° and {}-{} E° during {}'.format(plot_dic[variable]['title'],
                                                                          latz[i][1],latz[i][0],
                                                                          lonz[i][0],lonz[i][1],
                                                                                            month))
            fig.savefig(path+'Histogram_{}_lat_lower_left_{}_lon_lower_left_{}_{}_norm.png'.format(variable,latz[i][1],lonz[i][0],month))
            plt.close(fig)


In [23]:
plt.rcParams.update({'font.size': 12})

from matplotlib.path import Path
import matplotlib.patches as patches
path_file = '/media/onno/Algemeen/Thesis/GFS_mean_season/'
fig,axz=plt.subplots(1,3,figsize=(16,9))
variablez=['envelope','phasespeed','u_wind']
for j,ax in enumerate(axz.flat):
    print(j)
    ds = xr.open_dataset(path_file+'GFS_mean_error_{}_DJF_day_08_globe.nc'.format(variablez[j]),decode_times=False)
    ds=ds.sel(lat=slice(85,0))
    ds_cyclic, lon_cyclic = addcyclic(ds[plot_dic[variablez[j]]['data_variable']].values, ds.lon)
    lon,lat = np.meshgrid(lon_cyclic,ds.lat)
    latz = [(80,60),(60,40),(40,20)]
    lonz = [(320,340),(290,310),(320,340)]
    lonz = np.array(lonz)
    lonz_edit  = (lonz + 180) % 360 - 180
    m = Basemap(projection='ortho',lon_0=0,lat_0=90,resolution='c',ax=ax)
    # m = Basemap()
    # m.drawmapboundary(fill_color='aqua')
    # m.fillcontinents(color='coral',lake_color='aqua')
    m.drawcoastlines()
    m.drawmeridians((0,60,120,180,240,300),labels=[0,0,1,1],labelstyle='+/-')
    m.drawparallels((-80,-60,-40,-20,0,20,40,60,80))
    im=m.contourf(lon,lat,ds_cyclic,latlon='true', cmap=plt.cm.get_cmap('bwr'),
               vmin=-2,vmax=2,levels=np.linspace(-2,2,51),extend='both')

    for i in range(len(latz)):
        lbx1, lby1 = m(lonz[i][0], latz[i][1])
        ltx1, lty1 = m(lonz[i][0], latz[i][0])
        rtx1, rty1 = m(lonz[i][1], latz[i][0])
        rbx1, rby1 = m(lonz[i][1], latz[i][1])

        verts1 = [
            (lbx1, lby1), # left, bottom
            (ltx1, lty1), # left, top
            (rtx1, rty1), # right, top
            (rbx1, rby1), # right, bottom
            (lbx1, lby1), # ignored
            ]

        codes2 = [Path.MOVETO,
                 Path.LINETO,
                 Path.LINETO,
                 Path.LINETO,
                 Path.CLOSEPOLY,
                 ]

        path = Path(verts1, codes2)
        patch = patches.PathPatch(path, facecolor='none',edgecolor='maroon',lw=1.5)
        ax.add_patch(patch)
        ax.text(x=0.5,y=1.1,s=plot_dic[variablez[j]]['title'],
                   transform=ax.transAxes,horizontalalignment='center')

fig.suptitle('GEFS Reforecast seasonal Error DJF (+8 days) Compared to ERA5')
fig.subplots_adjust(left=0.03,right=0.97,bottom=0.2,top=0.9)
cbax = fig.add_axes([0.35, 0.1, 0.3, 0.025])
fig.colorbar(im, cax=cbax,ticks=np.linspace(-2,2,5),orientation='horizontal')
cbax.set_xlabel('Error (m/s)',rotation=0)

fig.show()

0


/home/onno/miniconda3/envs/thesis/lib/python3.8/site-packages/mpl_toolkits/basemap/__init__.py:5098: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  return npsel.concatenate((a,a[slicer]),axis=axis)


1


/home/onno/miniconda3/envs/thesis/lib/python3.8/site-packages/mpl_toolkits/basemap/__init__.py:5098: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  return npsel.concatenate((a,a[slicer]),axis=axis)


2


/home/onno/miniconda3/envs/thesis/lib/python3.8/site-packages/mpl_toolkits/basemap/__init__.py:5098: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  return npsel.concatenate((a,a[slicer]),axis=axis)


In [ ]:
latz = [(80,60),(60,40),(40,20)]
lonz = [(320,340),(290,310),(320,340)]